# Simulation methods: laws using accept-reject method

We here implement functions that generate samples following a specific law, from the random number generator.
This implementation will use the accept-reject method.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import factorial
from scipy.integrate import quad
%matplotlib inline

The `quad` function from the `scipy.integrate` module is used to compute the numerical integral of the law.

## Absolute value normal law

We first implement the absolute value normal law, which is defined as follows:
$$f(x)=\frac{2}{\sqrt{2\pi}}\exp\left(-\frac{x^2}{2}\right)\ \text{when}\ x>0\ \text{else}\ 0$$
As we don't want to generate negative values, we will use the accept-reject method.
We notice that:
$$\forall x>0,\qquad f(x)\leq\sqrt{\frac{2\mathrm e}\pi}\mathrm{e}^{-x}$$
Therefore, we set $c=\sqrt{\frac{2\mathrm e}\pi}$ and we generate $Y$ following an exponential law with $\lambda=1$.

In [ ]:
def exponentielle(lam, n):
    L=[]
    for i in range(n):
        u = np.random.random()
        L.append(-np.log(1-u)/lam)
    return L

def abs_norm(n):
    c = np.sqrt(2 * np.exp(1) / np.pi)
    L=[]
    while len(L) < n:
        u = np.random.random()
        g = exponentielle(1, 1)[0]
        f= 2 / np.sqrt(2 * np.pi) * np.exp(-g**2/2)
        h = f / (c * g)
        if u <= h:
            L.append(g)
    return L

X=abs_norm(10000)
plt.hist(X, density= True, ec = 'white')
f = lambda x: 2 / np.sqrt(2 * np.pi) * np.exp(-x**2/2)
plt.plot(np.linspace(0, 3, 100), f(np.linspace(0, 3, 100)))
plt.show()

## Law on $\mathbb R^2$

We now implement a law on $\mathbb R^2$, more specially around the unit circle.
For that, we will use the following algorithm:
1. Do `u1 <- 2 * rand - 1` and `u2 <- 2 * rand - 1` while `u1^2 + u2^2 > 1`.
2. `U1 <- u1` and `U2 <- u2`.

Then we plot the result, and also the unit circle, and a sqare of side 2 centered on the origin.

In [ ]:
def va(n = 10000):
    X=[]
    Y=[]
    for i in range(n):
        u1 = 2 * np.random.random() - 1
        u2 = 2 * np.random.random() - 1
        while u1 * u1 + u2 * u2 <= 1:
            u1 = 2 * np.random.random() - 1
            u2 = 2 * np.random.random() - 1
        X.append(u1)
        Y.append(u2)
    return X, Y

sample1, sample2 = va(1000)
u=0
v=0
a=1
b=1
t = np.linspace(0, 2*np.pi, 100)
plt.figure(figsize=(10, 10))
plt.scatter(sample1, sample2)
plt.plot(u + a * np.cos(t), v + b * np.sin(t), color='red')
plt.plot([-1, 1, 1 ,-1, -1], [1, 1, -1, -1, 1], color='green')
plt.axis('equal')
plt.show()

## Law on $\mathbb R^3$

We now consider a law on $\mathbb R^3$, more specially around the unit ball.
We will use a samilar algorithm as the previous one.

In [ ]:
def va_boule(n):
    L=[]
    while len(L) < n:
        x = -1 + 2 * np.random.random(3)
        if np.linalg.norm(x) <= 1:
            L.append(x)
    return L

sample = va_boule(1000)
ax = plt.figure().add_subplot(projection='3d')
ax.scatter([x[0] for x in sample], [x[1] for x in sample], [x[2] for x in sample])
plt.show()

## Gaussian binomial law

### Introduction

Using the accept-reject method, we can generate samples following a gaussian binomial law.
To do so, we will use the Box-Muller method to generate samples following a gaussian law.

In [ ]:
def gaussian2(n):
    r = np.array(exponentielle(0.5, n))
    u = 2 * np.pi * np.random.random(n)
    return np.sqrt(r) * np.cos(u), np.sqrt(r) * np.sin(u)

### Law on $\mathbb R^2$

We now consider a normal law on $\mathbb R^2$.

In [ ]:
X,Y = gaussian2(10000)
plt.figure(figsize=(10, 7))
plt.scatter(X,Y)
plt.show()

### Law on $\mathbb R$

We are going to use this method to generate a normal sample of size 10000, and then we plot the histogram of the result.
Thus, we can compare that with the normal law.

In [ ]:
gauss1 = gaussian2(10000)[0]
plt.figure(figsize=(10, 7))
plt.hist(gauss1, density= True, ec = 'white',label='Sample')
plt.plot(np.linspace(min(gauss1), max(gauss1), 100), 1/np.sqrt(2*np.pi) * np.exp(-np.linspace(min(gauss1), max(gauss1), 100)**2/2),label='Gaussian law')
plt.legend()
plt.show()

We notice that the histogram is not exactly the same as the normal law, but it is close enough.

### Gaussian binomial law on $\mathbb R^2$

Now, we are going to simulate a Gaussian binomial law.
For that, we set:
1. $Z_1=\mu_1+\sigma_{1,1}X_1+\sigma_{1,2}X_2$.
2. $Z_2=\mu_2+\sigma_{2,1}X_1+\sigma_{2,2}X_2$.

where $X_1$ and $X_2$ are two independent normal samples.
Then we define:
$$\mu=\binom{\mu_1}{\mu_2}$$
and
$$\sigma=\begin{pmatrix}\sigma_1^2&\rho\sigma_1\sigma_2\\\rho\sigma_1\sigma_2&\sigma_2^2\end{pmatrix}$$
with:
1. $\sigma_1=\sqrt{\sigma_{1,1}^2+\sigma_{1,2}^2}$.
2. $\sigma_2=\sqrt{\sigma_{2,1}^2+\sigma_{2,2}^2}$.
3. $\rho=\frac{\sigma_{1,1}\sigma_{2,1}+\sigma_{1,2}\sigma_{2,2}}{\sigma_1\sigma_2}$.

The law defined by $Z=(Z_1,Z_2)$ is a Gaussian binomial law with parameters $\mu$ and $\sigma$.

In [ ]:
def gaussian_binomial_density(z1, z2, sigma):
    sigma1 = np.linalg.norm(sigma1)
    sigma2 = np.linalg.norm(sigma2)
    rho = (sigma[1,1] * sigma[1,2] + sigma[2,1] * sigma[2,2]) / (sigma1 * sigma2)
    z = z1 ** 2 / sigma1 ** 2 + z2 ** 2 / sigma2 ** 2 - 2 * rho * z1 * z2 / (sigma1 * sigma2)
    return 1 / (2 * np.pi * sigma1 * sigma2 * np.sqrt(1 - rho ** 2)) * np.exp(-z / (2 * (1 - rho ** 2)))

In our case, we take $\mu_1=0$ and $\mu_2=0$.
We also want to have $\rho$ that takes the values $0.1$, $0.5$ and $0.9$.
We need to find values for the $\sigma_{i,j}$ in accordance with that.
To do so, we set $\sigma_{1,1}=1$ and $\sigma_{2,2}=1$.
Then we have:
1. $\sigma_{1,2}=0.01$.
2. $\sigma_{2,1}=0.09$.

In this situation, we have $\rho=0.1$.

To have $\rho = 0.5$, we set:
1. $\sigma_{1,2}=0.1$.
2. $\sigma_{2.1}=0.45$.

To have $\rho = 0.9$, we set:
1. $\sigma_{1,2}=0.5$.
2. $\sigma_{2.1}=0.77$.

In [ ]:
sigma11 = 1
sigma22 = 2
sigma_list = [(0.01,0.1), (0.1,0.42), (0.5,0.55)]

for s in sigma_list:
    sigma12, sigma21 = s
    sigma1 = np.linalg.norm([sigma11, sigma12])
    sigma2 = np.linalg.norm([sigma21, sigma22])
    rho = (sigma12 * sigma22 + sigma21 * sigma22) / (sigma1 * sigma2)
    sigma = np.array([[sigma1 ** 2, rho * sigma1 * sigma2], [rho * sigma1 * sigma2, sigma2 ** 2]])
    print(f'Computed rho = {rho}')
    print(f'Computed sigma = {sigma}')
    X1,X2 = gaussian2(10000)
    Z1 = sigma11 * X1 + sigma12 * X2
    Z2 = sigma21 * X1 + sigma22 * X2
    plt.title(f'rho = {rho}')
    plt.scatter(range(10000), Z2, label='Z2', color = 'blue')
    plt.scatter(range(10000), Z1, label='Z1', color = 'red')
    plt.legend()
    plt.show()
    plt.title(f'rho = {rho}, Z1')
    plt.hist(Z1, density= True, ec = 'white')
    plt.show()
    plt.title(f'rho = {rho}, Z2')
    plt.hist(Z2, density= True, ec = 'white')
    plt.show()

## Mixture of laws

We are looking for a way to represent a mixture of law.
Hence, we consider $f$ defined as follow:
$$f(x)=p_1f_1(x)+p_2f_2(x)+p_3f_3(x)$$
where $p_1=\frac16$, $p_2=\frac13$, $ p_3=\frac12$, and $f_i$ are laws.
In our case, we have:
1. $f_1(x)=1$ if $x\in[0,1]$ else $0$.
2. $f_2(x)=\frac12(2x-1)$ if $x\in]1,2]$ else $0$.
3. $f_3(x)=\frac23(-3x+9)$ if $x\in]2,3]$ else $0$.

We can first plot the function $f$, between $0$ and $3$.

In [ ]:
P = [1/6,1/3,1/2]

f1 = lambda x : 1 if 0 <= x <= 1 else 0
f2 = lambda x : 1/2*(2*x-1) if 1 < x <= 2 else 0
f3 = lambda x : 2/3*(-3*x+9) if 2 < x <= 3 else 0

def f(x):
    A=np.zeros(len(x))
    for i in range(len(x)):
        A[i] = P[0] * f1(x[i]) + P[1] * f2(x[i]) + P[2] * f3(x[i])
    return A

Using [`scipy.integrate.quad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.quad.html), we are going to check if this function is a density function.

In [ ]:
Pf1 = quad(lambda x : P[0] * f1(x), 0, 1)
Pf2 = quad(lambda x : P[1] * f2(x), 1, 2)
Pf3 = quad(lambda x : P[2] * f3(x), 2, 3)
print(f"Integral of f1 between 0 and 1: {Pf1[0]}, with error {Pf1[1]}")
print(f"Integral of f2 between 1 and 2: {Pf2[0]}, with error {Pf2[1]}")
print(f"Integral of f3 between 2 and 3: {Pf3[0]}, with error {Pf3[1]}")
print(f"Sum of integrals for integrating f between 0 and 3: {Pf1[0] + Pf2[0] + Pf3[0]}")
print(f"Maximum of error: {max(Pf1[1], Pf2[1], Pf3[1])}")

We are now going to plot this density function.

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(np.linspace(0, 3, 1000), f(np.linspace(0, 3, 1000)))
plt.show()

Using this density function, we are now going to generate samples following this law.
In our example, we want a sample of size $n=10000$.

In [ ]:
V = np.random.random(10000)

sample =[]
for v in V:
    if v <= Pf1[0]:
        for x in np.linspace(0,1,1000):
            if v >= quad(lambda x : P[0] * f1(x), 0, x)[0]:
                sample.append(x)
                break
    elif Pf1[0] < v <= Pf1[0] + Pf2[0]:
        for x in np.linspace(1,2,1000):
            if v >= Pf1[0] + quad(lambda x : P[1] * f2(x), 1, x)[0]:
                sample.append(x)
                break
    else:
        for x in np.linspace(2,3,1000):
            if v >= Pf1[0] + Pf2[0] + quad(lambda x : P[2] * f3(x), 2, x)[0]:
                sample.append(x)
                break

plt.hist(sample, density=True, ec='white')
plt.show()

# Gaussian mixture

We now consider a gaussian mixture, which is defined as follows:
$$f(x)=p_1f_1(x)+p_2f_2(x)$$
where $f_1$ and $f_2$ are two gaussian laws of parameters $(-3,1)$ and $(3,1)$, and $p_1$ and $p_2$ are two parameters.

In [ ]:
f1 = lambda x : 1 / np.sqrt(2 * np.pi) * np.exp(-(x + 3)**2/2)
f2 = lambda x : 1 / np.sqrt(2 * np.pi) * np.exp(-(x - 3)**2/2)

P = [(0.5,0.5),(1/4,3/4),(3/4,1/4)]

for p in P:
    plt.figure(figsize=(10,10))
    plt.plot(np.linspace(-10, 10, 100), f1(np.linspace(-10, 10, 100)), color='red',label='f1')
    plt.plot(np.linspace(-10, 10, 100), f2(np.linspace(-10, 10, 100)), color='blue', label='f2')
    plt.plot(np.linspace(-10, 10, 100), p[0] * f1(np.linspace(-10, 10, 100)) + p[1] * f2(np.linspace(-10, 10, 100)), color='green', label='f1 + f2')
    plt.legend()
    plt.show()

Using the $f$ function, we are now going to simulate a sample of size 10000 and plot their histogram.
This will be done for each tuple of parameters $(p_1,p_2)$.

In [ ]:
for p in P:
    mixed_sample = np.zeros(10000)

    for i in range(10000):
        u = np.random.random()
        if u <= p[0]:
            mixed_sample[i] = np.random.normal(-3, 1)
        else:
            mixed_sample[i] = np.random.normal(3, 1)

    plt.figure(figsize=(10,10))
    plt.hist(mixed_sample, density=True, ec='white')
    plt.show()